In [1]:
import geopandas as gpd
import pandas as pd
import glob
import numpy as np
from helper_functions import process_chunk_gadm, geometries_almost_equal

In [ ]:
reforestation_gdf = gpd.read_parquet('../midsave/updated_consolidated_reforestation_projects.parquet')

In [ ]:
reforestation_gdf = reforestation_gdf.set_crs(epsg=4326)

### Checking cases where polygon is an exact administration area

In [ ]:


# GADM dataset layers declaring
gadm_layers = ['ADM_0', 'ADM_1', 'ADM_2', 'ADM_3', 'ADM_4', 'ADM_5']  
gadm_data = {layer: gpd.read_file('/home/idisc02/Forest_Monitoring/input/gadm_410-levels.gpkg', layer=layer) for layer in gadm_layers}

# Processing each layer separately
for layer_name, gadm_data_layer in gadm_data.items():
    print(f"Processing layer: {layer_name}")
    
    reforestation_gdf[f'match_{layer_name}'] = 'No'
    
    # Processing the reforestation data in chunks
    chunk_size = 100000
    chunks = [reforestation_gdf[i:i + chunk_size] for i in range(0, len(reforestation_gdf), chunk_size)]
    
    for i, chunk in enumerate(chunks):
        print(f"Processing chunk {i + 1}/{len(chunks)} for layer {layer_name}")
        exact_matches_chunk = process_chunk_gadm(chunk, gadm_data_layer, layer_name)
        
        if not exact_matches_chunk.empty:
            # Marking the rows with exact matches as 'Yes'
            reforestation_gdf.loc[exact_matches_chunk.index, f'match_{layer_name}'] = 'Yes'
         
            exact_matches_chunk.to_file(f"exact_matches_{layer_name}_chunk_{i}.geojson", driver='GeoJSON')
        else:
            print(f"No matches found in chunk {i + 1} for layer {layer_name}")

    reforestation_gdf.to_file(f"reforestation_with_matches_{layer_name}.gpkg", driver='GPKG')

# Combining all exact matches for each layer into a single GeoDataFrame
for layer_name in gadm_layers:
    chunk_files = glob.glob(f"exact_matches_{layer_name}_chunk_*.geojson")
    exact_matches_list = [gpd.read_file(file) for file in chunk_files]
    
    if exact_matches_list:
        exact_matches_gdf = gpd.GeoDataFrame(pd.concat(exact_matches_list, ignore_index=True), crs=reforestation_gdf.crs)
        exact_matches_gdf.to_file(f"exact_matches_{layer_name}.gpkg", driver='GPKG')
    else:
        print(f"No exact matches found for layer {layer_name}")

print("Processing complete.")

In [ ]:
reforestation_gdf = reforestation_gdf.set_crs(epsg=4326, allow_override=True)
reforestation_gdf['exact-admin_area'] =reforestation_gdf[['match_ADM_0', 'match_ADM_1', 'match_ADM_2', 'match_ADM_3', 'match_ADM_4']].apply(lambda row: 'Yes' if 'Yes' in row.values else 'No', axis=1)
reforestation_gdf = reforestation_gdf.drop(columns=['match_ADM_1', 'match_ADM_2','match_ADM_0', 'match_ADM_3','match_ADM_4'])

### Checking if polygons are circle

In [ ]:




reforestation_gdf["circularity"] = (4 * np.pi * reforestation_gdf.area) / (reforestation_gdf.length ** 2)


circle_threshold = 0.98


reforestation_gdf["Polygon_acircle_oval"] = np.where(
   reforestation_gdf["circularity"] > circle_threshold, "Yes", "No"
)


reforestation_gdf["intersecting_circle"] = "No"


circle_oval_gdf = reforestation_gdf[reforestation_gdf["Polygon_acircle_oval"] == "Yes"]


for i, circle1 in circle_oval_gdf.iterrows():
    for j, circle2 in circle_oval_gdf.iterrows():
        if i >= j:
            continue  

       
        if circle1.geometry.intersects(circle2.geometry) or circle1.geometry.within(circle2.geometry) or circle2.geometry.within(circle1.geometry):
            
            reforestation_gdf.at[i, "intersecting_circle"] = "Yes"
            reforestation_gdf.at[j, "intersecting_circle"] = "Yes"


print(reforestation_gdf.info())
# print(reforestation_gdf)

In [ ]:

polygon_counts = reforestation_gdf["Polygon_acircle_oval"].value_counts()

intersecting_counts = reforestation_gdf["intersecting_circle"].value_counts()

print("Counts for 'Polygon_acircle_oval':")
print(polygon_counts)

print("\nCounts for 'intersecting_circle':")
print(intersecting_counts)

In [ ]:
reforestation_gdf.to_parquet("../midsave/consolidated_reforestation_projects.parquet")